In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import date, datetime, timedelta
import pickle
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
import seaborn as sns
from scipy import signal, stats
from scipy.stats import pearsonr

In [ ]:
turbine_readings = pd.read_csv("./data/WindData_scrubbed.csv")
weather_data = pd.read_csv("hourly_weather_data.csv")

In [ ]:
turbine_readings = turbine_readings.iloc[:, 0:74]
mean_readings = turbine_readings.apply(lambda x: np.mean(x), axis=1)
median_readings = turbine_readings.apply(lambda x: np.median(x), axis=1)
print(turbine_readings.shape)

In [ ]:
weather_data = weather_data.dropna(axis=1, how="all")
weather_data.fillna(method="ffill", inplace=True)
weather_data["timestamp"] = pd.to_datetime(weather_data["valid_time_gmt"], unit="s") - pd.Timedelta("08:00:00")
weather_data["weather_date"] = weather_data["timestamp"].dt.date
weather_data["weather_time"] = weather_data["timestamp"].dt.time
weather_data["weather_month"] = weather_data["timestamp"].dt.month
weather_data["weather_day_of_month"] = weather_data["timestamp"].dt.day
weather_data["weather_day_of_week"] = weather_data["timestamp"].dt.dayofweek
weather_data.set_index("timestamp", inplace=True)
windspeed = weather_data["wspd"]
windspeed_cubed = windspeed**3

In [ ]:
def plot_cross_correlation(feature, readings, interval):
    feature = feature.resample(interval).asfreq()
    feature.interpolate(method="linear", inplace=True)
    # signal.correlate calculates the integral(area) of the product of shifting time series
    cross_corrs_valid = signal.correlate(feature, readings, mode="valid", method="direct")
    print("Valid Shape: ", cross_corrs_valid.shape)
    cross_corrs_full = signal.correlate(feature, readings, mode="full", method="direct")
    print("Full Shape: ", cross_corrs_full.shape)
    #print(len(feature.index), cross_corrs_valid.shape)
    trace = go.Scatter(
        x = feature.index,
        y = cross_corrs_valid
    )
    data = [trace]
    py.iplot(data)
    plt.figure(figsize=(15,8))
    plt.plot(cross_corrs_full)
    plt.axvline(x=readings.shape[0]-1, color="red")
    plt.axvline(x=cross_corrs_full.shape[0]-readings.shape[0], color="red") 

In [ ]:
def plot_pearson_correlation(feature, readings, interval):
    feature = feature.resample(interval).asfreq()
    feature.interpolate(method="linear", inplace=True)
    # signal.correlate calculates the integral(area) of the product of shifting time series
    pears_corrs = pearsonr(feature, readings)
    print("pearsonr Shape: ", pears_corrs.shape)
    cross_corrs_full = signal.correlate(feature, readings, mode="full", method="direct")
    print("Full Shape: ", cross_corrs_full.shape)
    #print(len(feature.index), cross_corrs_valid.shape)
    trace = go.Scatter(
        x = feature.index,
        y = pears_corrs[0]
    )
    data = [trace]
    py.iplot(data)
    plt.figure(figsize=(15,8))
    plt.plot(cross_corrs_full)
    plt.axvline(x=readings.shape[0]-1, color="red")
    plt.axvline(x=cross_corrs_full.shape[0]-readings.shape[0], color="red") 

In [ ]:
plot_cross_correlation(windspeed_cubed, mean_readings, "1Min")

In [ ]:
plot_cross_correlation(windspeed_cubed, mean_readings, "5Min")

In [ ]:
plot_cross_correlation(windspeed_cubed, mean_readings, "20Min")

In [ ]:
plot_cross_correlation(windspeed_cubed, mean_readings, "1H")

In [ ]:
plot_cross_correlation(windspeed_cubed, mean_readings, "100Min")

In [ ]:
print(stats.pearsonr(turbine_readings.iloc[:, 0], turbine_readings.iloc[:, 1]))

In [ ]:
print(np.corrcoef(turbine_readings.iloc[:, 0], turbine_readings.iloc[:, 1]))

In [ ]:
windspeed_5min = windspeed_cubed.resample("5Min").asfreq()
windspeed_5min.interpolate(method="linear", inplace=True)
correlations = []
for i in range(0, (len(windspeed_5min)-len(mean_readings)+1)):
    correlations.append(stats.pearsonr(mean_readings, windspeed_5min.iloc[i:i+mean_readings.shape[0]])[0])

In [ ]:
print(len(correlations))
trace = go.Scatter(
    x = windspeed_5min.index,
    y = correlations
)
data = [trace]
py.iplot(data)

# Modelling the GE output (Regression)


In [ ]:
feature = windspeed_cubed
feature = feature.resample('5Min').asfreq()
feature.interpolate(method="linear", inplace=True)

In [ ]:
corr_array = signal.correlate(feature, mean_readings, mode='valid', method = 'direct')



In [ ]:
peak_timestamp = feature.index[np.argmax(corr_array)]

In [ ]:
peak_timestamp

In [ ]:
ranga = pd.date_range(start= peak_timestamp, periods = 2600, freq='5T')

In [ ]:
ranga

In [ ]:
time_turbine = pd.DataFrame(list(mean_readings), index=ranga)

In [ ]:
ge_data = time_turbine

In [ ]:
ge_data['wspd'] = feature[time_turbine.index]



In [ ]:
feature_temp = weather_data.temp
feature_temp = feature_temp.resample('5Min').asfreq()
feature_temp.interpolate(method="linear", inplace=True)
ge_data['temp'] = feature_temp[time_turbine.index]




In [ ]:
feature_wdir = weather_data.wdir
feature_wdir = feature_wdir.resample('5Min').asfreq()
feature_wdir.interpolate(method="linear", inplace=True)
ge_data['wdir'] = feature_wdir[time_turbine.index]




In [ ]:
ge_data.rename(columns = {0:'output'}, inplace=True)

In [ ]:
ge_data['dt_col'] = pd.to_datetime(ge_data.index)

In [ ]:
ge_data = ge_data.set_index('dt_col')

In [ ]:
ge_output = ge_data[['output']]



In [ ]:
ge_output = pd.Series(ge_data.output)

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)
#from plotly.offline import plot_mpl

#from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(ge_output, freq = 288)
fig = result.plot()
plot_mpl(fig)

#fig = go.Figure(d

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
#johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
coint_johansen(ge_data,-1,1).eig

In [ ]:
coint_johansen(ge_data[["output","wspd"]],-1,1).eig

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
minmax = MinMaxScaler(feature_range=(0,1))


ge_data.wspd = scaler.fit_transform(np.array(ge_data.wspd.values).reshape(-1,1)) 
ge_data.wdir = scaler.fit_transform(np.array(ge_data.wdir.values).reshape(-1,1))
ge_data.temp = scaler.fit_transform(np.array(ge_data.temp.values).reshape(-1,1)) 


ge_data.output = minmax.fit_transform(np.array(ge_data.output.values).reshape(-1,1))




In [ ]:
#creating the train and validation set
train = ge_data[:int(0.8*(len(ge_data)))]
valid = ge_data[int(0.8*(len(ge_data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model1 = VAR(endog=train)
model_fit = model1.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
# Baseline Model Random Forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:

param_grid = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, n_iter = 100, cv = 4, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train[['wspd','temp','wdir']], train.output)

In [ ]:
rf_random.score(valid[['wspd','temp','wdir']], valid.output)

In [ ]:
print("The RMSE for baseline RF model is: " ,np.sqrt(mean_squared_error( y_pred=rf_random.predict(valid[['wspd','temp','wdir']]), y_true=valid.output)))

In [ ]:
train_X = train[["wspd","temp","wdir"]]
train_y = train.output

val_X = valid[["wspd","temp","wdir"]]
val_y = valid.output

train_X = train_X.values
val_X = val_X.values

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
#test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))







from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU


# design network
model = Sequential()
#model.add(LSTM())

model.add(GRU(24, input_shape=(train_X.shape[1], train_X.shape[2])))
#model.add(GRU(100))


model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=75, batch_size=300, validation_data=(val_X, val_y), verbose=2, shuffle=False)
# plot history
#import matplotlib.pyplot as pyplot
#pyplot.plot(history.history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
#pyplot.legend()
#pyplot.show()


#y_pred = model.predict(test_X)

In [ ]:
y_pred = model.predict(val_X)

In [ ]:
cols = ge_data.columns

from sklearn.metrics import mean_squared_error


#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,3):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(pred[i], valid[i])))



In [ ]:
pred.output = pred.output.astype('float64')

In [ ]:
pred.output = np.array(pred.output).reshape(-1,)

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


#traces = []
#random_turbines = np.random.choice(turbine_names, size=3, replace=False)


trace1 = go.Scatter(x = valid.index,
                   y = list(100*val_y),
                   mode = 'lines',
                   name = 'test')


trace2 = go.Scatter(x = valid.index,
                   y = np.array(100*y_pred).reshape(-1,),
                   mode = 'lines',
                   name = 'pred')



traces=[trace1, trace2]



layout = go.Layout(
    title='GE data LSTM model'
    
)
fig = go.Figure(data=traces, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')

py.iplot(fig, filename='line-mode')

In [ ]:
np.sqrt(mean_squared_error(y_pred, val_y))

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


#traces = []
#random_turbines = np.random.choice(turbine_names, size=3, replace=False)


trace1 = go.Scatter(x = valid.index,
                   y = list(valid.output),
                   mode = 'lines',
                   name = 'test')


trace2 = go.Scatter(x = valid.index,
                   y = np.array(pred.output).reshape(-1,),
                   mode = 'lines',
                   name = 'pred')



traces=[trace1, trace2]



layout = go.Layout(
    title='VAR model'
    
)
fig = go.Figure(data=traces, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')

py.iplot(fig, filename='line-mode')

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


#traces = []
#random_turbines = np.random.choice(turbine_names, size=3, replace=False)


trace1 = go.Scatter(x = valid.index,
                   y = list(100*valid.output),
                   mode = 'lines',
                   name = 'Actual Output')


trace2 = go.Scatter(x = valid.index,
                   y = 100*rf_random.predict(valid[['wspd','temp','wdir']]),
                   mode = 'lines',
                   name = 'Predicted Output')



traces=[trace1, trace2]



layout = go.Layout(
    title='Random Forest model'
    
)
fig = go.Figure(data=traces, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')

py.iplot(fig, filename='line-mode')

# Sotavento Model

In [ ]:
import pandas as pd
#soluto_data = pd.read_csv('soluto_windfarm.csv', encoding='latin1')
#daily_soluto = pd.read_excel('soluto_daily.xlsx', sheet_name='daily')
hourly_soluto = pd.read_excel('soluto_daily.xlsx', sheet_name='hourly')

In [ ]:
#soluto_data.Date = pd.to_datetime(soluto_data.Date)
hourly_soluto.Date = pd.to_datetime(hourly_soluto.Date)

In [ ]:
hourly_soluto = hourly_soluto.sort_values("Date")

In [ ]:
hourly_soluto=hourly_soluto.set_index('Date')

In [ ]:
hourly_soluto.head()

In [ ]:
hourly_soluto = hourly_soluto[~hourly_soluto.index.duplicated(keep='first')]

In [ ]:
# Converting from european format to regular float

hourly_soluto['Energy'] = hourly_soluto['Energy'].apply(lambda x: x.replace('.','').replace(',', '.'))
hourly_soluto['Speed'] = hourly_soluto['Speed'].apply(lambda x: x.replace('.','').replace(',', '.'))
#hourly_soluto['Direction'] = hourly_soluto['Direction'].apply(lambda x: x.replace('.','').replace(",","."))

In [ ]:
hourly_soluto1 = hourly_soluto.replace({'-': 0.000001})

In [ ]:
hourly_soluto = hourly_soluto.replace({'-': np.nan})

In [ ]:
hourly_soluto_p = hourly_soluto.dropna()

In [ ]:
hourly_soluto_p.Energy = hourly_soluto_p.Energy.astype('float64')
hourly_soluto_p.Speed = hourly_soluto_p.Speed.astype('float64')

In [ ]:
subset = hourly_soluto['2013-01-01 00:00:00' : '2015-12-31 23:00:00']

In [ ]:
subset.Energy = subset.Energy.astype('float64')
subset.Speed = subset.Speed.astype('float64')

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)
#from plotly.offline import plot_mpl

#from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(subset.Speed.values, model='additive', freq = 24*365)
fig = result.plot()
plot_mpl(fig)

#fig = go.Figure(data=traces, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')

#py.iplot(fig, filename='line-mode')

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


data = [go.Scatter(x=hourly_soluto.index, y=hourly_soluto.Energy )]

py.iplot(data)

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


traces = []
#random_turbines = np.random.choice(turbine_names, size=3, replace=False)


for feature in soluto_data.columns[1:]:
    #filtered_turbine = daily_mean_turbine_reading.loc[daily_mean_turbine_reading["turbine_num"] == turbine]
    trace = go.Scatter(x = soluto_data.Date,
                       y = float(soluto_data[feature])/max(float(soluto_data[feature])),
                       mode = 'lines',
                       name = feature)
    
traces.append(trace)



layout = go.Layout(
    title='Time Series - Wind Speed (NOAA data), Turbine Output',
    yaxis=dict(
        title='Turbine Output (kWh)',
        #title='Wind Speed (mph)',
        titlefont=dict(
            color='blue'
        ),
        tickfont=dict(
            color='blue'
        ),
    ),
    yaxis2=dict(
        title='Wind Speed (mph)',
        titlefont=dict(
            color='orange'
        ),
        tickfont=dict(
            color='orange'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=traces, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')

py.iplot(fig, filename='line-mode')

In [ ]:
import statsmodels.api as sm
print(sm.tsa.stattools.grangercausalitytests(subset[['Energy','Speed']],maxlag=1000))

In [ ]:
print(sm.tsa.stattools.grangercausalitytests(subset[['Speed','Energy']],1))

## LSTM 

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, feat_names = None):
    n_vars = 1 if type(data) is list else data.shape[1]
    
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [(feat_names[j] + '(t-%d)' % i) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [(feat_names[j] + '(t)' ) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 



In [ ]:
values = dataset.values
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
val = hourly_soluto_p.values
scaler = StandardScaler()
scaled = scaler.fit_transform(val)

In [ ]:
reframed = series_to_supervised(scaled, n_in= 24,n_out= 1, feat_names= hourly_soluto_p.columns)

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = 365 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
X = hourly_soluto_p[['Speed', 'Direction']]
y = hourly_soluto_p[['Energy']]
X_train = X['2006-01-01 01:00:00' : '2016-12-30 23:00:00']
X_val = X['2017-01-01 01:00:00' : '2017-12-31 23:00:00']
X_test = X['2018-01-01 01:00:00' : '2018-11-30 23:00:00']


y_train = y['2006-01-01 01:00:00' : '2016-12-30 23:00:00']
y_val = y['2017-01-01 01:00:00' : '2017-12-31 23:00:00']
y_test = y['2018-01-01 01:00:00' : '2018-11-30 23:00:00']





In [ ]:
train_X = X_train.values
train_y = y_train.values

test_X = X_val.values
test_y = y_val.values



minmax = MinMaxScaler(feature_range=(0,1))

train_X = scaler.fit_transform(train_X)
train_y = minmax.fit_transform(train_y)

test_X = scaler.fit_transform(test_X) 
test_y = minmax.fit_transform(test_y) 

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))




In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU

In [ ]:
# design network
model = Sequential()
#model.add(LSTM())

model.add(GRU(75, input_shape=(train_X.shape[1], train_X.shape[2])))


model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=300, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
val_X = X_test.values
val_y = y_test.values



#minmax = MinMaxScaler(feature_range=(0,1))

val_X = scaler.fit_transform(val_X)
val_y = minmax.fit_transform(val_y)




In [ ]:
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))



In [ ]:
# make a prediction
y_pred = model.predict(val_X)



In [ ]:
pyplot.plot(pd.DataFrame(100*y_pred[1000:1200], index=  X_val.index[1000 : 1200]), label='Predction')
pyplot.plot(pd.DataFrame(100*val_y[1000:1200], index=  X_val.index[1000 : 1200]), label='Actual')
pyplot.legend()
pyplot.xlabel('time stamps')
pyplot.ylabel('% utilization')
pyplot.show()

In [ ]:
pyplot.plot(100*y_pred[4000:4200], label='Predction')
pyplot.plot(100*val_y[4000:4200], label='Actual')
pyplot.legend()
pyplot.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
100*mean_absolute_error(y_pred, val_y)

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

traces = []

#for turbine in random_turbines:
trace = go.Scatter(x = X_val.index,
                   y = y_pred,
                   mode = 'lines',
                   name = 'Prediction')
traces.append(trace)

py.iplot(traces, filename='line-mode')